In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib.font_manager as fm

[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name]
plt.rcParams['font.family'] = 'Malgun Gothic'

In [50]:
df_01 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_01_DEMO.csv')
df_02 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv')
df_03 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_03_COP_U.csv')
df_04 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_04_PD_CLAC.csv')
df_05 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_05_BR.csv')
df_06 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_06_LPAY.csv')

### 구매 기간 확인

In [5]:
df_01

,cust,ma_fem_dv,ages,zon_hlv
0,M000034966,여성,40대,Z07
1,M000059535,여성,30대,Z12
2,M000136117,여성,30대,Z11
3,M000201112,여성,50대,Z17
4,M000225114,여성,40대,Z17
...,...,...,...,...
29908,M999708287,여성,40대,Z10
29909,M999770689,여성,30대,Z16
29910,M999849895,여성,20대,Z04
29911,M999926092,남성,30대,Z08


In [6]:
df_02 = df_02[['cust', 'de_dt']]

In [7]:
df_03 = df_03[['cust', 'de_dt']]

In [8]:
df_06 = df_06[['cust', 'de_dt']]

In [9]:
df_02_03_06 = pd.concat([df_02,df_03,df_06], axis=0)
df_02_03_06

,cust,de_dt
0,M430112881,20210101
1,M646853852,20210101
2,M430112881,20210101
3,M430112881,20210101
4,M430112881,20210101
...,...,...
353179,M470112503,20210125
353180,M470112503,20210104
353181,M748878049,20210119
353182,M748878049,20210119


In [10]:
df_02_03_06['de_dt'] = df_02_03_06['de_dt'].astype('str')
df_02_03_06['de_dt'] = pd.to_datetime(df_02_03_06['de_dt'])
df_02_03_06

,cust,de_dt
0,M430112881,2021-01-01
1,M646853852,2021-01-01
2,M430112881,2021-01-01
3,M430112881,2021-01-01
4,M430112881,2021-01-01
...,...,...
353179,M470112503,2021-01-25
353180,M470112503,2021-01-04
353181,M748878049,2021-01-19
353182,M748878049,2021-01-19


In [67]:
def parallel_dataframe(df, func, cores):
    df_split = np.array_split(df, cores)
    pool = mp.Pool(cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    
    return df

In [68]:
from datetime import timedelta

def period_purchasing(cust, df):
    try:
        person_id = cust
        res = df[df['cust'] == person_id]['de_dt']
        res = sorted(list(set(res)))
        if len(res) > 1:
            ans = res[-1] - res[-2]
            ans = ans.days
        else:
            ans = 0

        return ans
    
    except: IndexError

In [82]:
from tqdm import tqdm

tqdm.pandas()

def make_period(data):
    data = df_01.progress_apply(lambda x : period_purchasing(x['cust'], data), axis=1)
    return data

In [83]:
import numpy as np
import multiprocessing as mp

df = parallel_dataframe(df_02_03_06, make_period, 6)

100%|█████████████████████████████████████| 29913/29913 [25:10<00:00, 19.80it/s]


In [88]:
df_01['P_period'] = df_01.progress_apply(lambda x : period_purchasing(x['cust'], df_02_03_06), axis=1)

100%|███████████████████████████████████| 29913/29913 [1:50:47<00:00,  4.50it/s]


In [90]:
df_01.isna().sum()

cust         0
ma_fem_dv    0
ages         0
zon_hlv      0
P_period     0
dtype: int64

In [89]:
df_01

,cust,ma_fem_dv,ages,zon_hlv,P_period
0,M000034966,여성,40대,Z07,54
1,M000059535,여성,30대,Z12,0
2,M000136117,여성,30대,Z11,14
3,M000201112,여성,50대,Z17,107
4,M000225114,여성,40대,Z17,6
...,...,...,...,...,...
29908,M999708287,여성,40대,Z10,58
29909,M999770689,여성,30대,Z16,13
29910,M999849895,여성,20대,Z04,47
29911,M999926092,남성,30대,Z08,76


In [91]:
df_01.to_csv('df_01_period(0805).csv')

In [143]:
df_01_period = pd.read_csv('./df_01_period.csv')

In [144]:
df_01_period = df_01_period[~df_01_period['P_period'].isna()]
df_01_period.drop('Unnamed: 0', axis=1, inplace=True)

In [147]:
df_01_period[df_01_period['P_period'] > 50]

,cust,ma_fem_dv,ages,zon_hlv,P_period
0,M000034966,여성,40대,Z07,54.0
3,M000201112,여성,50대,Z17,107.0
10,M000508243,여성,50대,Z17,60.0
18,M000803099,여성,50대,Z03,246.0
21,M000965026,남성,40대,Z17,66.0
...,...,...,...,...,...
29891,M999105944,남성,30대,Z10,61.0
29899,M999340261,남성,50대,Z11,77.0
29903,M999492154,여성,20대,Z10,256.0
29908,M999708287,여성,40대,Z10,58.0


### 구매 빈도

In [148]:
df_frequency = df_02['cust'].value_counts().to_frame()
df_frequency.reset_index(inplace=True)
df_frequency.columns = ['cust', 'count']
df_01_period = pd.merge(df_01_period, df_frequency, on='cust', how='inner')
df_01_period

,cust,ma_fem_dv,ages,zon_hlv,P_period,count
0,M000034966,여성,40대,Z07,54.0,51
1,M000136117,여성,30대,Z11,14.0,109
2,M000201112,여성,50대,Z17,107.0,21
3,M000225114,여성,40대,Z17,6.0,159
4,M000261625,여성,40대,Z17,29.0,67
...,...,...,...,...,...,...
25682,M999599111,여성,40대,Z04,22.0,34
25683,M999673157,남성,30대,Z10,2.0,24
25684,M999770689,여성,30대,Z16,13.0,315
25685,M999849895,여성,20대,Z04,47.0,72


In [155]:
df_01_period[df_01_period['P_period'] > 200]

,cust,ma_fem_dv,ages,zon_hlv,P_period,count
14,M000803099,여성,50대,Z03,246.0,2
52,M002631028,남성,40대,Z11,249.0,2
83,M003918629,여성,40대,Z10,224.0,23
293,M011970152,여성,50대,Z17,224.0,16
322,M013090322,여성,50대,Z17,241.0,1
...,...,...,...,...,...,...
25186,M979239892,남성,50대,Z16,236.0,7
25395,M988292446,남성,30대,Z16,216.0,5
25404,M988682724,남성,40대,Z10,279.0,6
25572,M995140494,여성,40대,Z10,231.0,12
